In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09.ipynb")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Lab 9: Time Series
### Using Zillow to Predict CPI
In this notebook, we're going to try to use data from Zillow (specifically, their ZORI index) to see if we can use it to predict the US consumer price index.

---
## Part 1: Introduction and Data Processing

### Dataset 1: ZORI

We've downloaded the ZORI index for you from [Zillow Research Data](https://www.zillow.com/research/data/). Run the following cell to display information about the ZORI index. What can you tell from the data?

In [ ]:
zori = pd.read_csv("zori1.csv")
zori

<!-- BEGIN QUESTION -->

**Question 1.1:** What information is stored in the ZORI data? What does each row represent?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 1.2:** If we wanted to compared ZORI data against US CPI, what might be some ways we can manipulate our data?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Below, we select row within the ZORI dataframe that has the timeseries corresponding to the ZORI index for the entire country.

In [ ]:
zUS = zori.iloc[[0], :-1]
zUS

We notice that the dates in the timeseries actually start from the 6th entry in the index. If we wanted to create a DataFrame, we would need to find a way to extract this information, in addition to the data stored in the ZORI column.

**Question 1.3:** Transpose the dataframe and select the relevant rows. Then rename the column names to `DATE` and `ZORI`. 

Hint: `df.T` can transpose a dataframe `df`. 
The resulting dataframe `zillow` should look like the following:

| DATE | ZORI |
| ----------- | ----------- |
| 2015-03-31 | 1370.301806 |
| 2015-04-30 | 1381.971304 |
|  | ... (rows omitted) |

In [ ]:
zillow = ... # transpose the dataframe `zUS`
zillow = ... # select relevant rows
zillow = zillow.reset_index()
zillow = ... # rename the columns
zillow['DATE'] = pd.to_datetime(zillow['DATE'], format='%Y-%m-%d') # convert date column to datatime objects
zillow

In [ ]:
grader.check("q1_3")

We might wonder what this looks like. Let's graph this time series.

In [ ]:
sns.lineplot(data=zillow, x="DATE", y="ZORI")
plt.title("ZORI Index");

Do you notice anything interesting about this graph? 

We'll focus on the part where the slope becomes noticably steeper. Can you find around what time this happened? What are some possible explanations?

### Dataset 2: US CPI

We'll now bring our attention to the US CPI.

We've downloaded historical **US CPI data for rent of primary residence** from the [FRED website](https://fred.stlouisfed.org/series/CUUR0000SEHA). Since our ZORI data starts on 03/31/2015, we've adjusted the sliders on the FRED website to only extract starting from 03/31/2015. We've also renamed the columns to be more readable.

In [ ]:
usCPI = pd.read_csv("usCPI.csv")
usCPI = usCPI.rename(columns={"CUUR0000SEHA": "CPI"})
usCPI = usCPI.iloc[1:, :]
usCPI['DATE'] = pd.to_datetime(usCPI['DATE'], format='%Y-%m-%d')
usCPI

We can try plotting this to see what it looks like.

In [ ]:
sns.lineplot(data=usCPI, x="DATE", y="CPI")
plt.title("US CPI (Rent of Primary Residence)");

### Data Preprocessing

In [ ]:
# set `DATE` as the index
zillow = zillow.set_index('DATE')
usCPI = usCPI.set_index('DATE')

**Question 1.4:** Let's split the `usCPI` dataframe into a training set and testing set. Put all data before Jan. 1st, 2022 into the training set, and all data on or after Jan. 1st, 2022 into the testing set. 

Note: This is not how normally we do the train-test split. In a normal setup, we would assign a random portion of the data to training set and the rest to testing set. But in this analysis, we care about how well can we predict the future values using current values, so here's how we do the train-test split. 

Hint: You can use `pd.to_datetime("2022-01-01", format='%Y-%m-%d')` to generate a timestamp for comparison. 

In [ ]:
train = ...
test = ...
plt.plot(train['CPI'], color = "black", label='Train')
plt.plot(test['CPI'], color = "red", label='Test')
plt.ylabel('')
plt.xlabel('Date')
plt.title("Train/Test split for US CPI Data")
plt.legend();

In [ ]:
grader.check("q1_4")

In [ ]:
# generate train test CPI series
y_train = train['CPI']
y_test = test['CPI']

---
## Part 2: Time Series Modeling

In [ ]:
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import warnings
warnings.filterwarnings("ignore")

### Review
Let $X_t$ be a sequence of random variables that describes a time series model. 

* **Random walk**:   
A random walk is a time series model where the current observation is equal to the previous observation with a random step up or down. The model is widely used for non-stationary data, particularly financial and economic data.

$$X_t = X_{t-1} + \varepsilon_t$$ 

* **Random walk with drift**:   
If the random walk has a non-zero dift and acts like a trend, we will add an additional drift term ($\alpha$) to the process. 

$$X_t = X_{t-1} + \alpha + \varepsilon_t$$ 

* **Autoregressive (AR)**: AR(p) refers to the autoregressive model of order p.  
The autoregressive model uses observations from preivous time steps as input to a regression equations to predict the value at the next step. The AR model takes in one argument, p, which determines how many previous time steps will be inputted.

$$X_t = c + \phi_1 X_{t-1} + \phi_2 X_{t-2} + \cdots + \phi_p X_{t-p} + \varepsilon_t$$ 

* **Moving-average (MA)**: MA(q) refers to the moving average model of order q. 
The moving average model is a time series model that accounts for very short-run autocorrelation. 

$$X_t = \mu + \varepsilon_t + \theta_1 \varepsilon_{t-1} + \theta_2 \varepsilon_{t-2} + \cdots + \theta_q \varepsilon_{t-q}$$ 

### Autoregressive Moving Average (ARMA)
The notation ARMA(p, q) refers to the model with p autoregressive terms and q moving-average terms. This model contains the AR(p) and MA(q) models. 

$$X_t = c + \underbrace{\phi_1 X_{t-1} + \phi_2 X_{t-2} + \cdots + \phi_p X_{t-p}}_{AR(p)} + \underbrace{\theta_1 \varepsilon_{t-1} + \theta_2 \varepsilon_{t-2} + \cdots + \theta_q \varepsilon_{t-q}}_{MA(q)} + \varepsilon_t $$ 

Here's a summary table for the relationship between ARIMA model and the models discussed. 

| Model | ARMA(p, q) |
| ----------- | ----------- |
| White noise | ARMA(0,0) |
| Autoregression (AR(p)) | ARMA(p,0) |
| Moving average (MA(q)) | ARMA(0,q) |

Now let's try fit our data using ARMA(1, 1), which is saying we want to fit our CPI data to the following process:

$$CPI_t = c + \underbrace{\phi_1 CPI_{t-1}}_{AR(1)} + \underbrace{\theta_1 \varepsilon_{t-1}}_{MA(1)} + \varepsilon_t $$ 

In [ ]:
ARMAmodel = ARIMA(y_train, order = (1, 0, 1)) # ARMA(1, 1). See ARIMA (next section) for why there's a zero
ARMAmodel = ARMAmodel.fit() 

In [ ]:
y_pred = ARMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) # 95% CI
y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_df

In [ ]:
# Get predictions series
y_pred_out = y_pred_df["Predictions"]

# Plot
plt.plot(train, color = 'black')
plt.plot(test, color = 'red', label = 'Actual')
plt.plot(y_pred_out, color='green', label = 'Predictions')
plt.ylabel('')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.title("Predictions US CPI Data")
plt.legend();

**Question 2.1:** Why does the predictions look almost constant?

1. Because ARMA is a bad model for this time series
2. Because we did not give ARMA a reasonable set of orders (i.e. p and q)
3. Because the data is too volatile to predict

Assign the number corresponding to your answer to `q2_1` below.

In [ ]:
q2_1 = ...

In [ ]:
grader.check("q2_1")

We can also compute the loss of this model. 

In [ ]:
arma_rmse = np.sqrt(mean_squared_error(test["CPI"].values, y_pred_df["Predictions"]))
print("RMSE: ",arma_rmse)

### Autoregressive Integrated Moving Average (ARIMA)

The notation ARIMA(p, d, q) refers to the model with p autoregressive terms with d degree of differencing and q moving-average terms. 

$$X_t' = c + \underbrace{\phi_1 X_{t-1}' + \phi_2 X_{t-2}' + \cdots + \phi_p X_{t-p}'}_{AR(p)} + \underbrace{\theta_1 \varepsilon_{t-1} + \theta_2 \varepsilon_{t-2} + \cdots + \theta_q \varepsilon_{t-q}}_{MA(q)} + \varepsilon_t $$ 

where $y_t'$ is the differenced series (it may have been differenced more than once).

**Difference between ARMA and ARIMA**: The only difference is the “integrated” part. Integrated refers to the number of times needed to difference a series in order to achieve stationarity, which is required for ARMA models to be valid. So ARMA is a special case of ARIMA with $d = 0$. ARIMA models are widely used for real life time series analysis since most times series data are non stationary and need differencing.

Here's a summary table for the relationship between ARIMA model and the models discussed. 

| Model | ARIMA(p, d, q) |
| ----------- | ----------- |
| White noise | ARIMA(0,0,0) |
| Random walk | ARIMA(0,1,0) with no constant |
| Random walk with drift | ARIMA(0,1,0) with a constant |
| Autoregression (AR(p)) | ARIMA(p,0,0) |
| Moving average (MA(q)) | ARIMA(0,0,q) |
| Autoregressive Moving Average (ARMA(p,q)) | ARIMA(p,0,q) |

Now let's try fit our data using $\text{ARIMA}(1, 1, 1)$. 

In [ ]:
ARIMAmodel_naive = ARIMA(y_train, order = (1, 1, 1))
ARIMAmodel_naive = ARIMAmodel_naive.fit()

y_pred = ARIMAmodel_naive.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARIMAmodel_naive.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"] 
plt.plot(train, color = "black")
plt.plot(test, color = "red", label = 'Actual')
plt.plot(y_pred_out, color='blue', label = 'ARIMA(1, 1, 1) Predictions')
plt.legend();

In [ ]:
arima_naive_rmse = np.sqrt(mean_squared_error(test["CPI"].values, y_pred_df["Predictions"]))
print("RMSE: ",arima_naive_rmse)

After adding $d=1$ differencing, the predictions look much better than before!

However, the ARIMA model for time series analysis and forecasting can be tricky to configure.

There are 3 parameters (p, d and q) that require estimation by iterative trial and error from reviewing diagnostic plots and using 40-year-old heuristic rules. Usually we will find optimal parameters with ACF, PACF plots, and BIC/AIC information criteria or simply do a grid search, but we will not go into depth about these topics here. If you are interested, you can learn more about this in Stat 153. 

**Question 2.2:** Let's find some better parameters using trial and error. Try some combinations of p, d and q and see what is the result of the predictions. To pass the autograder, you only need to get RMSE of your model under 3.5. 

Hint: Think about what does p, d, q mean in the ARIMA model. 

In [ ]:
ARIMAmodel = ... # specify the model
ARIMAmodel = ... # fit the model

# generate predictions
y_pred = ARIMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"] 

# plot
plt.plot(train, color = "black")
plt.plot(test, color = "red", label = 'Actual')
plt.plot(y_pred_out, color='blue', label = 'ARIMA Predictions')
plt.legend();

In [ ]:
arima_rmse = np.sqrt(mean_squared_error(test["CPI"].values, y_pred_df["Predictions"]))
print("RMSE: ",arima_rmse)

In [ ]:
grader.check("q2_2")

However, in real world when we don't know the future values, such tuning can lead to overfitting. So we need to be aware of the bia-variance tradeoff when tuning our time series model. 

### Seasonal Autoregressive Integrated Moving Average (SARIMA)
So far, we have restricted our attention to non-seasonal data and non-seasonal ARIMA models. However, ARIMA models are also capable of modelling a wide range of seasonal data.

A seasonal ARIMA model is formed by including additional seasonal terms in the ARIMA models we have seen so far. It is written as follows:

$$\text{ARIMA} (p, d, q) (P, D, Q)_m$$

where $m$ is the number of observations per year. For example, $m=12$ is for monthly data and $m=4$ is for quarterly data. 

**Question 2.3:** What should $m$ be for our dataset?

In [ ]:
m = ...

In [ ]:
grader.check("q2_3")

Let's try fit our data using $\text{ARIMA} (5, 4, 2) (2, 2, 2)_{m}$. 

In [ ]:
SARIMAXmodel = SARIMAX(y_train, order = (5, 4, 2), seasonal_order=(2,2,2,m))
SARIMAXmodel = SARIMAXmodel.fit(disp=0)

In [ ]:
y_pred = SARIMAXmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"] 
plt.plot(train, color = "black")
plt.plot(test, color = "red", label = 'Actual')
plt.plot(y_pred_out, color='Blue', label = 'SARIMA Predictions')
plt.legend()

In [ ]:
sarima_rmse = np.sqrt(mean_squared_error(test["CPI"].values, y_pred_df["Predictions"]))
print("RMSE: ",sarima_rmse)

**Question 2.4:** It seems that the SARIMA model doesn't help too much with our predictions. Why?

1. Because SARIMA is not an improvement to ARIMA for this time series
2. Because we did not give SARIMA a reasonable set of orders
3. Because the data is too volatile to predict

Assign the number corresponding to your answer to `q2_4` below.

In [ ]:
q2_4 = ...

In [ ]:
grader.check("q2_4")

---
## Part 3: Time Series with Multiple Series

We want to combine both time series so that we can analyze them together. However, we notice that the `DATE` column has different values. ZORI is measured on the 31st of each month, while CPI is measured on the 1st day of each month. For our purposes, we can combine the two by using the `DATE` index from the US CPI and using the 1st day of the month.

In [ ]:
combined = pd.concat([zillow.reset_index(drop=True), usCPI.reset_index(drop=True)], axis=1)
combined.index = usCPI.index
combined

### Visual Comparison

We can try plotting this graph to see if we can find anything interesting.

In [ ]:
sns.lineplot(data=combined, x="DATE", y="CPI", color='tab:blue', label='CPI')
ax2 = plt.twinx()
sns.lineplot(data=combined, x="DATE", y="ZORI", color='tab:orange', ax=ax2, label='ZORI')
plt.legend(loc='upper left')
ax2.legend(loc='lower right')
plt.title("US CPI vs ZORI");

#### Logging Our Data

We can try logging our data to make them easier to compare.

**Question 3.1:** Take the natural log for `CPI` and `ZORI` and store the results in new columns `log CPI` and `log ZORI`. Then make a plot similar to the one above. 

Hint: You may want to convert both column `ZORI` and `CPI` to `float64` before taking the natural log. 

In [ ]:
...
combined["log CPI"] = ...
...
combined["log ZORI"] = ...

sns.lineplot(..., color='tab:blue', label='log CPI')
plt.ylim([5.6, 6.1])
plt.legend(loc='upper left')

ax2 = plt.twinx()
sns.lineplot(..., color='tab:orange', ax=ax2, label='log ZORI')
ax2.set_ylim([7.2, 7.7])
ax2.legend(loc='lower right')
plt.title("US CPI vs ZORI (Logged)");

In [ ]:
grader.check("q3_1")

<!-- BEGIN QUESTION -->

**Question 3.2:** Which time series is more volatile? What can be one possible explanation?

_Type your answer here, replacing this text._

<!-- END QUESTION -->



In [ ]:
# correlation
rho, pvalue = stats.pearsonr(combined["log CPI"], combined["log ZORI"])
print("Correlation: ", rho)
print("P-value: ", pvalue)

### Vector Autoregressive (VAR) Models

One limitation of the models that we have considered so far is that they impose a unidirectional relationship — the forecast variable is influenced by the predictor variables, but not vice versa. However, there are many cases where the reverse should also be allowed for — where all variables affect each other. For example, in our case CPI and ZORI can be predictor for each other since they are highly correlated series.  

A VAR model is a generalization of the univariate autoregressive model for forecasting a vector of time series. We will not go into the math behind VAR model in this class. 

In [ ]:
# Import Statsmodels packages
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [ ]:
var_df = combined[["CPI", "ZORI"]]
var_df

#### Splitting into Training and Testing Groups

In [ ]:
nobs = 4
df_train, df_test = var_df[0:-nobs], var_df[-nobs:]

# Check size
print(df_train.shape) 
print(df_test.shape)

In [ ]:
df_train.dtypes

---
### Step 1: Checking Correlations

#### Granger Causality
We first use Granger's Causality Tests to see the p-value of the correlations between different variables.

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [ ]:
grangers_causation_matrix(var_df, variables = var_df.columns) 

<!-- BEGIN QUESTION -->

**Question 3.3:** How can we interpret the findings above?

Hint: Read the docstring for the function `grangers_causation_matrix`. 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

#### Cointegration Test
We can also use a cointegration test to see if the two time series are correlated before making any predictions.

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

In [ ]:
cointegration_test(var_df)

### Step 2: Transforming the Time Series to be Stationary
In order to perform our VAR test, we want our time series to be stationary. A stationary time series is one whose characteristics like mean and variance does not change over time. This is similar to what we have done with ARIMA model. 

Augmented Dickey Fuller test (ADFuller or ADF Test) is a common statistical test used to test whether a given Time series is stationary or not. It is one of the most commonly used statistical test when it comes to analyzing the stationary of a series.

In [ ]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")  

We will keep differencing the data series until we pass the ADFuller test for stationarity. 

#### First Iteration (with no differencing)

In [ ]:
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

#### Second Iteration

In [ ]:
df_differenced = df_train.diff().dropna()

In [ ]:
# ADF Test on each column of 1st Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

#### Third Iteration

In [ ]:
df_differenced = df_differenced.diff().dropna()

In [ ]:
# ADF Test on each column of 2nd Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

Now it seems that both series are stationary. 

### Step 3: Fitting the Model

In [ ]:
model = VAR(df_differenced)
for i in [1,3,5,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

In [ ]:
model_fitted = model.fit(3)
model_fitted.summary()

In [ ]:
from statsmodels.stats.stattools import durbin_watson

def adjust(val, length= 6): return str(val).ljust(length)
out = durbin_watson(model_fitted.resid)

for col, val in zip(df_differenced.columns, out):
    print(adjust(col), ':', round(val, 2))

### Step 4: Making Predictions

In [ ]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
forecast_input

In [ ]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=var_df.index[-nobs:], columns=var_df.columns + '_2d')
df_forecast

These are the differenced results. We want to de-difference it to get it back in terms of the original.

In [ ]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [ ]:
df_results = invert_transformation(df_train, df_forecast, second_diff=True)        
df_results

We plot our predictions against actual values.

In [ ]:
fig, axes = plt.subplots(nrows=int(len(var_df.columns)/2), ncols=2, dpi=150, figsize=(12,6))
for i, (col,ax) in enumerate(zip(var_df.columns, axes.flatten())):
    df_results[col+'_forecast'].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    df_test[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actual")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

### Step 5: Finding RMSE

In [ ]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: ZORI')
accuracy_prod = forecast_accuracy(df_results['ZORI_forecast'].values, df_test['ZORI'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

**Congrats on making it to the end of the lab!**

---
## Feedback

**Question 4:** Please fill out this short [feedback form](https://forms.gle/VwKv8efSwPwYNQve8) to let us know your thoughts about this lab! We really appreciate your opinions and feedback! At the end of the Google form, you should see a codeword. Assign the codeword to the variable `codeword` below. 

In [ ]:
codeword = ...

In [ ]:
grader.check("q4")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)